# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set_style('darkgrid')
%matplotlib inline

In [2]:
df = pd.read_csv('homepage_actions.csv')
df.head()


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
views = df[df['action']=='view']
click = df[df['action']=='click']

In [4]:
#sorting by actions taken by unique ids
df_actions = df.groupby(['id','group'])['action'].apply(list).reset_index()
max_actions = df_actions.action.map(lambda x: len(x)).max()
df_actions[['action1','action2']] = pd.DataFrame(df_actions.action.values.tolist(), index= df_actions.index)
df_actions.head()

,id,group,action,action1,action2
0,182988,experiment,[view],view,None
1,182994,control,"[view, click]",view,click
2,183089,control,[view],view,None
3,183136,experiment,[view],view,None
4,183141,experiment,"[view, click]",view,click


In [5]:
#Sorting by groups each Id falls into
df_group_overlap = df.groupby(['id'])['group'].apply(list).reset_index()
max_groups = df_actions.group.map(lambda x: len(x)).max()
df_group_overlap['number_groups'] = df_group_overlap['group'].apply(lambda x: len(x))
df_group_overlap[['group1','group2']] = pd.DataFrame(df_group_overlap.group.values.tolist(), index= df_group_overlap.index)
df_group_overlap.head()

,id,group,number_groups,group1,group2
0,182988,[experiment],1,experiment,None
1,182994,"[control, control]",2,control,control
2,183089,[control],1,control,None
3,183136,[experiment],1,experiment,None
4,183141,"[experiment, experiment]",2,experiment,experiment


In [6]:
# checking for users that appear in control and test
repeat_id = []
for i in range(0,len(df_group_overlap)):
    if len(df_group_overlap.group[i]) == 2:
        if df_group_overlap.group[i][0] != df_group_overlap.group[i][1]:
            repeat_id.append(df_group_overlap.id[i])
repeat_id

[]

In [7]:
print('Summary of Initial findings')
print("There are {} records".format(len(df)))
print("There are {} unique IDs".format(len(df.id.unique())))
print("There are {} users that either took more than one action or are in both the experiment and control group".format(len(df)-len(df.id.unique())))
print("*************************")
print("There are {} unique actions a user can take".format(len(df.action.unique())))
print("The unique actions are: {} - {}".format(df.action.unique()[0],df.action.unique()[1]))
print('The max number of actions taken by one user is: {}'.format(max_actions))
print("*************************")
print("There are {} users that viewed the homepage".format(len(views)))
print("That represent {}% of all users".format(round(len(views)/len(df.id.unique())*100,2)))
print("*************************")
print("There are {} users that click on homepage".format(len(click)))
print("That represent {}% of all users".format(round(len(click)/len(df.id.unique())*100,2)))
print("There is no contamination between the control and Experiment")



Summary of Initial findings
There are 8188 records
There are 6328 unique IDs
There are 1860 users that either took more than one action or are in both the experiment and control group
*************************
There are 2 unique actions a user can take
The unique actions are: view - click
The max number of actions taken by one user is: 2
*************************
There are 6328 users that viewed the homepage
That represent 100.0% of all users
*************************
There are 1860 users that click on homepage
That represent 29.39% of all users
There is no contamination between the control and Experiment


In [8]:
# Finalizing the dataframe

In [9]:
#cleaning groups
df_group_clean = df_group_overlap.drop(labels=['group','number_groups','group2'], axis=1)
df_group_clean.head()

#cleaning actions
df_actions_clean = df_actions.drop(labels=['group','action'], axis=1)
df_actions_clean.head()

#cleaing time 
from datetime import datetime

df_time_stamp = df.groupby(['id'])['timestamp'].apply(list).reset_index()
df_time_stamp[['time1','time2']] = pd.DataFrame(df_time_stamp.timestamp.values.tolist(), index= df_group_overlap.index)
df_time_stamp_clean = df_time_stamp.drop(labels='timestamp',axis=1)
df_time_stamp_clean['time1'] = pd.to_datetime(df_time_stamp_clean['time1'], infer_datetime_format=True)
df_time_stamp_clean['time2'] = pd.to_datetime(df_time_stamp_clean['time2'], infer_datetime_format=True)
df_time_stamp_clean['diff'] = df_time_stamp_clean['time2'] - df_time_stamp_clean['time1']



In [10]:
#joining 
df_final = df_group_clean.set_index('id').join(df_actions_clean.set_index('id'),rsuffix='_join').join(df_time_stamp_clean.set_index('id'),rsuffix='_join')
df_final.reset_index(inplace=True)
df_final.head()

,id,group1,action1,action2,time1,time2,diff
0,182988,experiment,view,None,2016-11-26 14:38:47.682931,NaT,NaT
1,182994,control,view,click,2016-12-14 13:19:00.940677,2016-12-14 13:20:29.564080,00:01:28.623403
2,183089,control,view,None,2016-09-29 23:04:42.722325,NaT,NaT
3,183136,experiment,view,None,2016-12-27 10:13:58.808014,NaT,NaT
4,183141,experiment,view,click,2016-11-20 07:58:52.768641,2016-11-20 08:00:22.434253,00:01:29.665612


In [11]:
df_final['count_click'] = [1.0 if x == 'click' else 0.0 for x in df_final['action2']]

In [12]:
# creating both test groups
df_control = df_final[df_final['group1'] == "control"]
df_test = df_final[df_final['group1'] == 'experiment']
df_control_CTR = round(df_control.count_click.sum()/len(df_control)*100,2)
df_test_CTR = round(df_test.count_click.sum()/len(df_test)*100,2)
clicks_control = df_control.count_click.sum()
clicks_test = df_test.count_click.sum()

print('There are {} ids in the control sample'.format(len(df_control)))
print('There are {} ids in the experiment sample'.format(len(df_test)))

There are 3332 ids in the control sample
There are 2996 ids in the experiment sample


In [13]:
# summary Stats
print('Summary Stats')
print('*****************')
print('Clicks \tControl:{} \tTest:{}'.format(clicks_control,clicks_test))
print('CTR: \tControl:{}% \tTest:{}%'.format(df_control_CTR, df_test_CTR))

Summary Stats
*****************
Clicks 	Control:932.0 	Test:928.0
CTR: 	Control:27.97% 	Test:30.97%


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

### Hypothesis
H_0 = The experimental site does not drive more engagement

H_1 = The experimental site drives more clicks then the control group

In [14]:
p_scypi = stats.ttest_ind(df_control.count_click, df_test.count_click, equal_var=False)[1]/2
p_scypi

0.004466402814337101

In [15]:
print('Given the results of our test there is enough evidence to reject the null hypothesis that the experimental site did not drive more clicks')


Given the results of our test there is enough evidence to reject the null hypothesis that the experimental site did not drive more clicks


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [16]:
#Your code here
expected_clicks = round(len(df_test)*df_control_CTR/100,0)
print('With no effect we would expect to see {} clicks'.format(expected_clicks))

With no effect we would expect to see 838.0 clicks


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [17]:
#Your code here
var = len(df_test)*df_control_CTR/100*(1 - df_control_CTR/100)
std = round(np.sqrt(var),2)
print('The actual results are {} standard deviations removed from the estimate'.format(std))

The actual results are 24.57 standard deviations removed from the estimate


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [18]:
#Your code here
import scipy.stats as stats

z = (clicks_test - expected_clicks)/std
p_val = stats.norm.sf(z) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012463748635944125


In [19]:
print()

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: While we obtain a slighly lower p-value using the verification  method, both lead to the rejection of the null hypothesis 
As such, assuming a 95% confidence interval we can be 95% certain that the experimental page had an affect on the CTR of the home page

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.